<a href="https://colab.research.google.com/github/nscotto/gpu_ml/blob/master/CUBLAS_Housing_Skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This exercise uses the housing data available by default on Colab. For a description of this data, see [here](https://developers.google.com/machine-learning/crash-course/california-housing-data-description). The example is losely based on Google's (excellent) course on [logistic regression](https://colab.research.google.com/notebooks/mlcc/logistic_regression.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=logisticregression-colab&hl=en#scrollTo=B5OwSrr1yIKD).

The data is stored in the CSV file `sample_data/california_housing_test.csv`. The target is in column
`median_house_value`. The goal is to use the remaining columns to estimate whether the target is above 265000 (label `1`) or below (label `0`).

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!pip install --upgrade git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-yc3rzxq_
  Stored in directory: /tmp/pip-ephem-wheel-cache-uer6_p9d/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

In [0]:
%%cu 
/** CUBLAS example taken from http://courses.cms.caltech.edu/cs179/ */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#include <iostream>
#include <iomanip>
#include <math.h>
#include <fstream>

/* Includes, cuda */
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <helper_cuda.h>

using namespace std;


/* transform matrix index to vector offset
   Since CUDA uses column major, 
   ld = number of rows 
   Example of use: a[IDX2C(0, 1, 50)] */
#define IDX2C(i,j,ld) (((j)*(ld))+(i))

/* Constants for housing data set */
#define data_columns  (9)
#define above_threshold (265000.0)

/////////////////////////////////////////////////////////
// Number of rows in arrays to print for debugging
/////////////////////////////////////////////////////////
#define print_rows (10)

/////////////////////////////////////////////////////////
// Auxiliary function
/////////////////////////////////////////////////////////
// generate random numbers in interval [min,max]
float float_rand( float min, float max )
{
    float scale = rand() / (float) RAND_MAX; /* [0, 1.0] */
    return min + scale * ( max - min );      /* [min, max] */
}


/////////////////////////////////////////////////////////
// Functions for reading the dataset from a file
/////////////////////////////////////////////////////////

/* Read a csv file with a given number of rows and columns */
void read_csv(const char* filename, float* data_array,size_t nbrow,size_t nbcol) {
  string row_as_string;
  string value;
  double ioTemp;
  ifstream infile;
  infile.open(filename, ifstream::in);
  size_t row_count = 0;
	if (infile.is_open())
  {
      // read the headers (and discard)
			getline(infile, row_as_string, '\n');
      cout << "headers: " << row_as_string << "!" << std::endl;
      for(int i = 0; i < nbrow; i++){
  			getline(infile, row_as_string, '\n');
        // cout << "read line " << row_as_string << "!" << std::endl;
				istringstream line_stream(row_as_string);
			  for(int j = 0; j < nbcol; j++){
          getline(line_stream, value, ',');
					ioTemp = strtod(value.c_str(), NULL); 
          // cout << "("<<i<<","<<j<<") = "<< ioTemp << std::endl;

					data_array[IDX2C(i,j,nbrow)] = ioTemp;

				}
        ++row_count;
			}
		infile.close();
    cout << "Read " << row_count << " rows." << std::endl;
	}
	else cout << "Cannot open file." << endl;
}

/* Allocate memory and fill with given number of rows from housing file */
void read_housing_csv(float** data_array, size_t nbrows) {
    *data_array = (float *)malloc(nbrows * data_columns * sizeof(float));
    
    read_csv("sample_data/california_housing_test.csv",*data_array,nbrows,data_columns);

    cout << "Data (first "<<print_rows<<" rows):" << std::endl;
    // Show some entries for double checking
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < data_columns; j++){
				cout << (*data_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}

/////////////////////////////////////////////////////////
// Functions for preprocessing the data set
/////////////////////////////////////////////////////////

/* Split data into inputs and labels. Allocated memory for inputs and labels.
   Since cuBLAS is column major, each input is in a column.
   We also add 1.0 as first element to each input vector.
*/
void get_inputs_and_labels(float* data_array, float** input_array, float** label_array, size_t nbrows, size_t nbcols, size_t* nb_inputs, size_t* nb_labels ) {
    // The inputs are the first nbrows-1 columns.
    // The labels are the last column (index nbrows-1), booleanized
    // by the condition >= above_threshold
    size_t nbcols_input = nbcols-1+1; // remove one column, add one for 1.0
    size_t nbcols_labels = 2;
    *nb_inputs = nbcols_input;
    *nb_labels = nbcols_labels;
    *input_array = (float *)malloc(nbrows * nbcols_input * sizeof(float));    
    *label_array = (float *)malloc(nbrows * nbcols_labels * sizeof(float));    
    //cout << &input_array << " and "<< &label_array << " data " << data_array << std::endl;
    cout << "Allocated memory: " << nbrows << " rows, "<< nbcols_input << " columns." << std::endl;

    // Copy the data to X
    for(int i = 0; i < nbrows; i++){
      // Set the first element of each x to 1  
      (*input_array)[IDX2C(0,i,nbcols_input)] = 1.0;
      // Copy the rest of x
			for(int j = 1; j < nbcols_input; j++){
				(*input_array)[IDX2C(j,i,nbcols_input)] = data_array[IDX2C(i,j-1,nbrows)];
			}
      float median_house_value = data_array[IDX2C(i,nbcols-1,nbrows)];
      (*label_array)[IDX2C(i,0,nbrows)] = 0.0;
      (*label_array)[IDX2C(i,1,nbrows)] = 0.0;
      if (median_house_value >= above_threshold) {
        (*label_array)[IDX2C(i,0,nbrows)] = 1.0;
      } else {
        (*label_array)[IDX2C(i,1,nbrows)] = 1.0;        
      }
		}    
    
    // Show some entries for double checking
    cout << "Inputs (first "<<print_rows<<"):" << std::endl;
	  for(int j = 0; j < nbcols_input; j++){
      for(int i = 0; i < nbrows && i < print_rows; i++){
				cout << (*input_array)[IDX2C(j,i,nbcols_input)] << "\t";
			}
      cout << "\n";
		}
    cout << "Labels (first "<<print_rows<<"):" << std::endl;
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < nbcols_labels; j++){
				cout << (*label_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}


/////////////////////////////////////////////////////////
// Main program
/////////////////////////////////////////////////////////
int main(int argc, char **argv)
{
    
    /////////////////////////////////////////////////////////
    // Parameters for the data set
    /////////////////////////////////////////////////////////
    size_t N_train = 2000; // points for training (Google: 12000)
    size_t N_test = 1000; // points for validation (Google: 5000)
    size_t N = N_train+N_test;
    /////////////////////////////////////////////////////////
    // Reading the data set
    /////////////////////////////////////////////////////////
    float *alldata = 0;
    read_housing_csv(&alldata,N);
    float* X = 0; 
    float* Y = 0;
    size_t D; // number of inputs
    size_t M; // number of labels
    get_inputs_and_labels(alldata,&X,&Y,N,data_columns,&D,&M);
    /////////////////////////////////////////////////////////
    // Inputs and labels are now available in X and Y.
    // Each input is a column in X; X is of dimension D x N
    // each label is a row in Y; Y is of dimension N x M
    /////////////////////////////////////////////////////////

    
    /////////////////////////////////////////////////////////
    // Parameters for Stochastic Gradient Descent
    /////////////////////////////////////////////////////////
    int nb_iter = 10;
    int periods = nb_iter; // reporting periods
    float step_size = 1.0;
    
    
    /////////////////////////////////////////////////////////
    // Memory Allocation and Initialization
    /////////////////////////////////////////////////////////
    cublasStatus_t status;
    float *h_W;
    float *h_Z;
    float *h_q;
    float *d_X = 0;
    float *d_W = 0;
    float *d_Z = 0;
    float *d_q = 0;
    float alpha = 1.0f;
    float beta = 0.0f;
    size_t nX = D * N; // nb of elements in X
    size_t nW = D * M; // nb of elements in W
    size_t nZ = M * N; // nb of elements in Z=W^TX
    int i,j,k;
    float J;
    cublasHandle_t handle;

    status = cublasCreate(&handle);

    /* Allocate host memory for the matrices */
    h_Z = (float *)malloc(nZ * sizeof(h_Z[0]));
    h_q = (float *)malloc(M * sizeof(h_q[0]));

    /////////////////////////////////////////////////////////
    // Initializing Weight Matrix 
    // its dimension is D x M
    /////////////////////////////////////////////////////////
    h_W = (float *)malloc(nW * sizeof(h_W[0]));
    for (i = 0; i < nW  ; ++i) {
        h_W[i] = float_rand( -1, 1 );
    }

    /* Allocate device memory for the matrices */
    cudaMalloc((void **)&d_Z, nZ * sizeof(d_Z[0]));
    cudaMalloc((void **)&d_X, nX * sizeof(d_X[0]));
    cudaMalloc((void **)&d_W, nW * sizeof(d_W[0]));
    cudaMalloc((void **)&d_q, M * sizeof(d_q[0]));

    /* Initialize the device matrices with the host matrices */
    status = cublasSetVector(nX, sizeof(X[0]), X, 1, d_X, 1);
    status = cublasSetVector(nW, sizeof(h_W[0]), h_W, 1, d_W, 1);

    /////////////////////////////////////////////////////////
    // Batch Gradient Descent
    /////////////////////////////////////////////////////////
    for (i = 0; i < nb_iter; ++i ) {
        ////////////////////////////////
        // compute Z = W^T X'
        // --> each column z of Z corresponds to one column x' of X'
        ////////////////////////////////
        // Reminder: 
        // W^T has dimensions M by D
        // X' has dimensions D by N
        // Z has dimensions M by N
        
        /* ... to be completed using cublasSgemm */

        ////////////////////////////////
        // Compute softmax activation p(z)
        // Version 1: on the host
        ////////////////////////////////
        // Copy Z back to host
        status = cublasGetVector(nZ, sizeof(h_Z[0]), d_Z, 1, h_Z, 1);
          
        ////////////////////////////////
        // For each column z of Z, compute p;
        // then update W
        ////////////////////////////////
        J = 0.0;
        for (j=0; j < N_train; ++j) {
          ////////////////////////////////
          // For the k-th element of z,
          // compute p_k = exp(z_k)/sum_t exp(z_t)
          //         q_k = p_k - y_k(j)
          ////////////////////////////////
        
          /* ... compute q_k and store in array h_q[k] ...
            ... to be completed ... */
            
          // Copy q onto device
          status = cublasSetVector(M, sizeof(h_q[0]), h_q, 1, d_q, 1);
            
          ////////////////////////////////
          // Compute the LogLoss score
          // J = - sum_j sum_k y_k * log(p_k)
          ////////////////////////////////
            
          /* ... to be completed ... */

          ////////////////////////////////
          // compute dW = a x'(p-y)^T;
          // update   W = W - a dW
          ////////////////////////////////
  
          /* ... to be completed using cublasSgemm */
          
          // Note: If you get an error messages concerning SGEMM 
          //       (instead of cublasSgemm), know that cublasSgemm calls
          //       SGEMM as follows:
          // SGEMM(TRANSA, TRANSB, M, N, K, ALPHA, A, LDA, B, LDB, BETA, C, LDC)
        }
        
        // Compute the classification score
        if (i%(nb_iter/periods)==0) {
          printf("iter: %d, logloss: %f\n",i,J);
        }
    }
    
    /* Read the result back */
    status = cublasGetVector(nW, sizeof(h_W[0]), d_W, 1, h_W, 1);

    /* Evaluate the accuracy */
    int evaluate_accuracy = 1;
    if (evaluate_accuracy) {
        
        /* ... apply classifier on training set ...
           ... to be completed ... */

        /* ... compute and print accuracy on training set ...
           ... to be completed ... */

    }
    
    /* Memory clean up */
    free(h_Z);
    free(h_W);
    free(h_q);

    cudaFree(d_q);
    cudaFree(d_X);
    cudaFree(d_W);
    cudaFree(d_Z);

    /* Shutdown */
    status = cublasDestroy(handle);
}


UsageError: Cell magic `%%cu` not found.
